In [22]:
from decouple import config
import together
from langchain import PromptTemplate, LLMChain
from typing import Any, Dict, List, Mapping, Optional
#pydantic model version>2 will not work with langchain. you should install a model version less than 2
from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env
import re

together.api_key = config("TOGETHER_API_KEY")

together.Models.start("togethercomputer/llama-2-70b-chat")

class TogetherLLM(LLM):
    """Together large language models."""

    model = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key = config("TOGETHER_API_KEY")
    """Together API key"""

    temperature = 0.7
    """What sampling temperature to use."""

    max_tokens = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self):
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt,
        **kwargs: Any,
    ):
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        # Use regex substitution to remove newlines
        #cleaned_text = re.sub(r"\n", "", text)
        return text

tog_llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=2000)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template





In [18]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(file_path='./reviews_list.csv', encoding="utf-8", csv_args={'delimiter': ','})
text = loader.load()

25


I got the prompt from here : https://github.com/FreedomIntelligence/Evaluation-of-ChatGPT-on-Information-Extraction/blob/main/4_ABSA/prompt/absa_prompt.json


In [19]:
system_prompt = "You are an advanced assistant. You are always helpful in times of research "
instruction =  """
              find out the reviews from the text.Recognize all aspect terms with their corresponding sentiment polarity in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. Determine the sentiment polarity from the options [\"positive\", \"negative\", \"neutral\"]. Answer in the format [\"aspect\", \"sentiment\"] without any explanation. If no aspect term exists, then only answer \"[]\". "
               ```{text}```
           """
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt= prompt, llm = tog_llm )

[INST]<<SYS>>
You are an advanced assistant. You are always helpful in times of research 
<</SYS>>


              find out the reviews from the text.Recognize all aspect terms with their corresponding sentiment polarity in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. Determine the sentiment polarity from the options ["positive", "negative", "neutral"]. Answer in the format ["aspect", "sentiment"] without any explanation. If no aspect term exists, then only answer "[]". "
               ```{text}```
           [/INST]


In [23]:
reviews = llm_chain.run(text)

print(reviews)

 Here are the reviews with their corresponding sentiment polarity:

1. "inconvenient" - negative
2. "great" - positive
3. "unbearable" - negative
4. "great" - positive
5. "horrible bleach smell" - negative
6. "Loved room decor" - positive
7. "Construction noise" - negative
8. "quality of renovations" - negative
9. "Regular house keeping" - negative
10. "Request for ketttle and iron ignored" - negative
11. "Expensive for quality of hotel" - negative
12. "Horrible wifi" - negative
13. "Sauna/ location" - positive
14. "Valet staff were lovely" - positive
15. "it wasn’t set up for more than a one night stay" - negative
16. "We had a mini kitchen in our room that didn’t have paper towels or dish soap" - negative
17. "They only do cleaning on request and they don’t replace essentials like toilet paper" - negative
18. "The cleaning staff did not do the dishes either" - negative
19. "Great location!" - positive
20. "Prime location in the heart of Waikiki close to everything" - positive
21. "Th

In [24]:
system_prompt = "You are an advanced assistant. You are always helpful in times of research "
instruction =  """
              make a report from the reviews. calculate the positive, negative and neutral reviews and make a decision for me, whether should I stay on that hotel or not? reviews is inside the triple back quotes.
               ```{reviews}```
           """
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["reviews"])

llm_chain = LLMChain(prompt= prompt, llm = tog_llm )

report = llm_chain.run(reviews)

print(report)

[INST]<<SYS>>
You are an advanced assistant. You are always helpful in times of research 
<</SYS>>


              make a report from the reviews. calculate the positive, negative and neutral reviews and make a decision for me, whether should I stay on that hotel or not? reviews is inside the triple back quotes.
               ```{reviews}```
           [/INST]
 Sure, I can help you with that.

Based on the reviews you provided, here's a breakdown of the positive, negative, and neutral reviews:

Positive Reviews: 24
Negative Reviews: 31
Neutral Reviews: 6

Based on the majority of negative reviews, it seems that the hotel may not be the best choice for you. Many reviewers mentioned that the hotel is overpriced, the rooms are small and outdated, and the bathrooms are not well-maintained. Additionally, some reviewers mentioned that the hotel's location is convenient, but also noisy due to road noise and homeless people yelling at night.

On the other hand, some reviewers mentioned that t